# STEP 2: Define Evaluation Metrics
* PSNR
* SSIM

In [1]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt

def plt_img(input_array, model_x, w, h):
    input_img = input_array[0]
    input_img = input_img.transpose((1, 2, 0))
    
    if model_x == "SRCNN":
        plt.imshow(input_img)
    elif model_x == "ESPCN":
        input_img_resize = cv2.resize(input_img, (w, h), interpolation = cv.INTER_AREA)
        plt.imshow(input_img_resize)
    plt.show()

def PSNR(original, compressed):
    mse = np.mean( (original/255. - compressed/255.) ** 2 )
    if mse < 1.0e-10: return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def calculate_PSNR(original_batch, compressed_batch, batch_size):
    PSNR_TOTAL = 0
    for num in range(batch_size):
        psnr = PSNR(original_batch[num], compressed_batch[num])
        PSNR_TOTAL = PSNR_TOTAL + psnr
    return PSNR_TOTAL

def SSIM(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_SSIM(img1, img2):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

# STEP 3: Design Custom Dataset

In [2]:
import cv2 as cv
import os
from torch.utils.data import Dataset, DataLoader

class DIV2K_Dataset(Dataset):
    
    def __init__(self, width, height, scale, path_to_imgs, model_x, transform = None):
        self.model_x = model_x
        self.width = width
        self.height = height
        self.scale = scale
        self.path_to_imgs = path_to_imgs
        self.length = len(os.listdir(path_to_imgs))
        self.transform = transform
        
    def __getitem__(self, index):
        # Interpolation: INTER_CUBIC, INTER_NEAREST, INTER_LINEAR, INTER_LANCZOS4, 【INTER_AREA】
        img = cv.imread(self.path_to_imgs + os.listdir(self.path_to_imgs)[index])
        img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        h, w, c = img_rgb.shape
        if (h > w):
            img_rgb = img_rgb.transpose((1,0,2))
        
        img_hr   = cv.resize(img_rgb,   (self.width             , self.height             ), interpolation = cv.INTER_AREA)
        img_lr_1 = cv.resize(img_hr ,   (self.width //self.scale, self.height //self.scale), interpolation = cv.INTER_AREA)
        img_lr_2 = cv.resize(img_lr_1 , (self.width             , self.height             ), interpolation = cv.INTER_AREA)
        if self.transform and self.model_x == "SRCNN":
            img_lr_tensor = self.transform(img_lr_2)
            img_hr_tensor = self.transform(img_hr)
        
        elif self.transform and self.model_x == "ESPCN":
            img_lr_tensor = self.transform(img_lr_1)
            img_hr_tensor = self.transform(img_hr)
            
        return (img_lr_tensor, img_hr_tensor)
    
    def __len__(self):
        return self.length

# STEP 4: Define Model ==> SRCNN
* Github Repo Link: https://github.com/yjn870/SRCNN-pytorch
* Difference:
    1. Added the zero padding
    2. Used the Adam instead of the SGD
    3. Removed the weights initialization

In [3]:
from torch import nn

class SRCNN(nn.Module):
    def __init__(self, num_channels = 3):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size = 9, padding = 9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size = 5, padding = 5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size = 5, padding = 5 // 2)
        self.relu = nn.ReLU(inplace = True)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x
    
    
class ESPCN(nn.Module):
    def __init__(self, scale_factor, num_channels=2):
        super(ESPCN, self).__init__()
        self.first_part = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=5, padding=5//2),
            nn.Tanh(),
            nn.Conv2d(64, 32, kernel_size=3, padding=3//2),
            nn.Tanh(),
        )
        self.last_part = nn.Sequential(
            nn.Conv2d(32, num_channels * (scale_factor ** 2), kernel_size=3, padding=3 // 2),
            nn.PixelShuffle(scale_factor)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.in_channels == 32:
                    nn.init.normal_(m.weight.data, mean=0.0, std=0.001)
                    nn.init.zeros_(m.bias.data)
                else:
                    nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
                    nn.init.zeros_(m.bias.data)

    def forward(self, x):
        x = self.first_part(x)
        x = self.last_part(x)
        return x

# STEP 5: Summerize Model & Set DataLoader

In [4]:
from torchsummary import summary
from torchvision import transforms
import torch

width, height = 2000, 1600
scale = 4
path_to_train_imgs = "../dataset/Flickr2K/"
path_to_valid_imgs = "../dataset/DIV2K/DIV2K_valid_HR/"
trans_train = transforms.Compose([transforms.ToTensor()])
trans_valid = transforms.Compose([transforms.ToTensor()]) 
batch_size = 6
model_x = "ESPCN"

Train_Dataset = DIV2K_Dataset(width = width, height = height, scale = scale, path_to_imgs = path_to_train_imgs, model_x = model_x, transform = trans_train)
Train_Dataloader = DataLoader(Train_Dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

Valid_Dataset = DIV2K_Dataset(width = width, height = height, scale = scale, path_to_imgs = path_to_valid_imgs, model_x = model_x, transform = trans_valid)
Valid_Dataloader = DataLoader(Valid_Dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

if False and model_x == "SRCNN":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SRCNN(num_channels = 3)
    model = model.to(device)
    summary(model, input_size = (3, width, height))  
elif False and model_x == "ESPCN":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ESPCN(scale_factor = scale, num_channels = 3)
    model = model.to(device)
    summary(model, input_size = (3, width//scale, height//scale))
    # Estimated total size (MB): 220.60

# STEP 6: Set Hyper Parameter

In [5]:
import torch
from torch import nn
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if model_x == "SRCNN":
    model = SRCNN(num_channels = 3).to(device)
elif model_x == "ESPCN":
    model = ESPCN(scale_factor = scale, num_channels = 3).to(device)
    
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
epochs = 10

# STEP 7: Train

In [ ]:
from tqdm import tqdm
import os

#load weight
def load_checkpoint(model, checkpoint_PATH, optimizer):
    model_CKPT = torch.load(checkpoint_PATH)
    model.load_state_dict(model_CKPT['model_state_dict'])
    print('loading checkpoint!')
    optimizer.load_state_dict(model_CKPT['optimizer_state_dict'])
    return model, optimizer

#checkpoint_PATH = 'checkpoints/epoch2_0.0003274347081536516_12.9556.pth'
#model,optimizer = load_checkpoint(model, checkpoint_PATH, optimizer)

probe_number =  80
loss_list = []
train_psnr_before_list = []
train_psnr_after_list = []
valid_psnr_before_list = []
valid_psnr_after_list = []

for epoch in range(epochs):
    
    model.train()
    running_loss = 0
    running_psnr_before = 0
    running_psnr_after = 0
    inner_epoch_count = 0
    train_total = 0
    for data in tqdm(Train_Dataloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(inputs.shape)
        preds = model(inputs).clamp(0.0, 1.0)
        
        loss = criterion(preds, labels)
        if model_x == "SRCNN":
            psnr_before= calculate_PSNR(inputs.data.cpu().numpy(), labels.data.cpu().numpy(), labels.size(0))
            running_psnr_before = running_psnr_before + psnr_before

        psnr_after = calculate_PSNR( preds.data.cpu().numpy(), labels.data.cpu().numpy(), labels.size(0))
        running_psnr_after  = running_psnr_after  + psnr_after

        running_loss = running_loss + loss.item()
        
        train_total = train_total + labels.size(0)
        inner_epoch_count = inner_epoch_count + 1
        if inner_epoch_count % probe_number == probe_number - 1:
            for ele in [inputs.data.cpu().numpy(), preds.data.cpu().numpy(), labels.data.cpu().numpy()]:
                plt_img(ele, model_x,width, height)
            loss_list.append(running_loss/train_total)
            if model_x == "SRCNN":
                train_psnr_before_list.append(running_psnr_before/train_total)
            
            train_psnr_after_list.append(running_psnr_after/train_total)
            print(f"Before: {round(running_psnr_before/train_total, 4)}, After: {round(running_psnr_after/train_total, 4)}, Diff: {(round((running_psnr_after - running_psnr_before)/train_total, 4))}, Loss: {round(running_loss/train_total, 5)}")
            train_total = 0
            running_psnr_before = 0
            running_psnr_after = 0
            running_loss = 0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    running_psnr_before = 0
    running_psnr_after = 0
    valid_total = 0
    for data in tqdm(Valid_Dataloader):
        inputs, labels = data[0].to(device), data[1].to(device)
        valid_total = valid_total + labels.size(0)
        with torch.no_grad():
            preds = model(inputs).clamp(0.0, 1.0)
                
        if model_x == "SRCNN":
            psnr_before= calculate_PSNR(inputs.data.cpu().numpy(), labels.data.cpu().numpy(), labels.size(0))
            valid_psnr_before_list.append(psnr_before/labels.size(0))
            running_psnr_before = running_psnr_before + psnr_before/labels.size(0)
        
        psnr_after = calculate_PSNR( preds.data.cpu().numpy(), labels.data.cpu().numpy(), labels.size(0))
        valid_psnr_after_list.append(psnr_after/labels.size(0))
        running_psnr_after = running_psnr_after + psnr_after/labels.size(0)
    torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),}, 
               os.path.join('./checkpoints', 'epoch{0}_{1}_{2}.pth'.format(epoch,running_loss/train_total,round(running_psnr_after/valid_total, 4))))   

    print(f"Before: {round(running_psnr_before/valid_total, 4)}, After: {round(running_psnr_after/valid_total, 4)}")
            

  0%|▎                                                                                 | 2/525 [00:01<07:25,  1.17it/s]

# STEP 8: Plot

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_psnr_before_list)
plt.plot(train_psnr_after_list)
plt.show()

#plt.plot(valid_psnr_before_list)
plt.plot(valid_psnr_after_list)
plt.show()

plt.plot(loss_list)
plt.show()

In [7]:
from torch import nn

class ResidualBlock(nn.Module):
  def __init__(self, channels):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(channels, channels, kernel_size = 3, padding = 1)
    self.bn1 = nn.BatchNorm2d(channels)
    self.prelu = nn.PReLU()
    self.conv2 = nn.Conv2d(channels, channels, kernel_size = 3, padding = 1)
    self.bn2 = nn.BatchNorm2d(channels)

  def forward(self, x):
    residual = self.conv1(x)
    residual = self.bn1(residual)
    residual = self.prelu(residual)
    residual = self.conv2(residual)
    residual = self.bn2(residual)

    return x + residual

class UpsampleBlock(nn.Module):
  def __init__(self, in_channels, up_scale):
    super(UpsampleBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, in_channels * up_scale ** 2, kernel_size = 3, padding = 1)
    self.pixel_shuffle = nn.PixelShuffle(up_scale)
    self.prelu = nn.PReLU()
  
  def forward(self, x):
    x = self.conv(x)
    x = self.pixel_shuffle(x)
    x = self.prelu(x)
    return x

In [8]:
import torch
from torch import nn
from torchvision.models.vgg import vgg16

class GeneratorLoss(nn.Module):
  def __init__(self):
    super(GeneratorLoss, self).__init__()
    vgg = vgg16(pretrained = True)
    loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
    for param in loss_network.parameters():
      param.requires_grad = False
    self.loss_network = loss_network
    self.mse_loss = nn.MSELoss()
    self.tv_loss = TVLoss()

  def forward(self, out_labels, out_images, target_images):
    # Adversarial Loss
    adversarial_loss = torch.mean(1 - out_labels)
    # Perception Loss
    perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
    # Image Loss
    image_loss = self.mse_loss(out_images, target_images)
    # TV Loss
    tv_loss = self.tv_loss(out_images)
    return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss

class TVLoss(nn.Module):
  def __init__(self, tv_loss_weight = 1):
    super(TVLoss, self).__init__()
    self.tv_loss_weight = tv_loss_weight
  
  def forward(self, x):
    batch_size = x.size()[0]
    h_x = x.size()[2]
    w_x = x.size()[3]
    count_h = self.tensor_size(x[:, :, 1:, :])
    count_w = self.tensor_size(x[:, :, :, 1:])
    h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
    w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
    return self.tv_loss_weight * 2 * (h_tv/count_h + w_tv/count_w)/batch_size

  @staticmethod
  def tensor_size(t):
    return t.size()[1] * t.size()[2] * t.size()[3]

In [9]:
import math
import torch
from torch import nn

class Generator(nn.Module):
  def __init__(self, scale_factor):
    upsample_block_num = int(math.log(scale_factor, 2))

    super(Generator, self).__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 9, padding = 4),
        nn.PReLU()
    )
    self.block2 = ResidualBlock(64)
    self.block3 = ResidualBlock(64)
    self.block4 = ResidualBlock(64)
    self.block5 = ResidualBlock(64)
    self.block6 = ResidualBlock(64)
    self.block7 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(64)
    )
    block8 = [UpsampleBlock(64, 2) for _ in range(upsample_block_num)]
    block8.append(nn.Conv2d(64, 3, kernel_size=9, padding=4))
    self.block8 = nn.Sequential(*block8)
  
  def forward(self, x):
    block1 = self.block1(x)
    block2 = self.block2(block1)
    block3 = self.block3(block2)
    block4 = self.block4(block3)
    block5 = self.block5(block4)
    block6 = self.block6(block5)
    block7 = self.block7(block6)
    block8 = self.block8(block1 + block7)

    return (torch.tanh(block8) + 1) / 2

In [15]:
import torch
from torch import nn
import PIL.Image as pil_image
import numpy as np
import torch.backends.cudnn as cudnn
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim_cal

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def img_preprocess(image_file):
    img = cv.imread(image_file)
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    h, w, c = img_rgb.shape
    width, height = 2000, 1600

    if (h > w):
        img_rgb = img_rgb.transpose((1,0,2))
    img_hr   = cv.resize(img_rgb,   (width             , height             ), interpolation = cv.INTER_AREA)
    img_lr_1 = cv.resize(img_hr ,   (width //scale, height //scale), interpolation = cv.INTER_AREA)
    img_lr_2 = cv.resize(img_lr_1 , (width             , height             ), interpolation = cv.INTER_AREA)
    #img_hr   = cv.resize(img_rgb,   (width             , height             ), interpolation = cv.INTER_LINEAR)
    #img_lr_1 = cv.resize(img_hr ,   (width //scale, height //scale), interpolation = cv.INTER_LINEAR)
    #img_lr_2 = cv.resize(img_lr_1 , (width             , height             ), interpolation = cv.INTER_LINEAR)
    image_in = transforms.ToTensor()(img_lr_2).unsqueeze(0).to(device)
    image_lr = transforms.ToTensor()(img_lr_1).unsqueeze(0).to(device)
    image_hr = transforms.ToTensor()(img_hr).unsqueeze(0).to(device)
    return image_in,image_lr,image_hr

scale = 4
model_SRCNN = SRCNN(num_channels = 3).to(device)
model_ESPCN = ESPCN(scale_factor = scale, num_channels = 3).to(device)
model_SRGAN = Generator(scale).to(device)

cudnn.benchmark = True



#load SRCNN checkpoint
checkpoint_SRCNN = 'checkpoints/epoch9_0.00033620840335953913_13.0112.pth'
model_CKPT_SRCNN = torch.load(checkpoint_SRCNN)
model_SRCNN.load_state_dict(model_CKPT_SRCNN['model_state_dict'])
model_SRCNN.eval()
#load ESPCN checkpoint
checkpoint_ESPCN = 'checkpoints/epoch9_0.000404924631030077_12.8325.pth'
model_CKPT_ESPCN = torch.load(checkpoint_ESPCN)
model_ESPCN.load_state_dict(model_CKPT_ESPCN['model_state_dict'])
model_ESPCN.eval()
#load SRGAN checkpoint
checkpoint_SRGAN = '../epochs/netG_epoch_4_13.pth'
model_CKPT_SRGAN = torch.load(checkpoint_SRGAN)
model_SRGAN.load_state_dict(model_CKPT_SRGAN)
model_SRGAN.eval()

#image_file = '../dataset/set5/img_001.png'
img_path_set5 = '../dataset/set5'
f1 = os.listdir(img_path_set5)
ssim_r_set5 = 0
psnr_r_set5 = 0
ssim_srcnn_set5 = 0
psnr_srcnn_set5 = 0
ssim_espcn_set5 = 0
psnr_espcn_set5 = 0
ssim_srgan_set5 = 0
psnr_srgan_set5 = 0
for image_file in f1:
    
    image_in,image_lr,image_hr = img_preprocess(img_path_set5+'/'+image_file)
    
    ####RESIZE ONLY
    psnr_resize = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*image_in.squeeze(0).cpu().numpy().transpose([1, 2, 0]))
    ssim_resize = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*image_in.squeeze(0).cpu().numpy().transpose([1, 2, 0]))
    psnr_r_set5 += psnr_resize
    ssim_r_set5 += ssim_resize
    ####SRCNN
    result_SRCNN = model_SRCNN(image_in).clamp(0.0, 1.0)
    psnr_srcnn = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRCNN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_srcnn = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRCNN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_srcnn_set5 += psnr_srcnn
    ssim_srcnn_set5 += ssim_srcnn
    ####ESPCN
    result_ESPCN = model_ESPCN(image_lr).clamp(0.0, 1.0)
    psnr_espcn = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_ESPCN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_espcn = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_ESPCN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_espcn_set5 += psnr_espcn
    ssim_espcn_set5 += ssim_espcn
    ####SRGAN
    result_SRGAN = model_SRGAN(image_lr).clamp(0.0, 1.0)
    psnr_srgan = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRGAN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_srgan = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRGAN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_srgan_set5 += psnr_srgan
    ssim_srgan_set5 += ssim_srgan
    
    
    print('Set5: '+image_file)
    print('      RESIZE PSNR:'+str(psnr_resize)+' SSIM:'+str(ssim_resize))
    print('       SRCNN PSNR:'+str(psnr_srcnn)+' SSIM:'+str(ssim_srcnn))
    print('       ESPCN PSNR:'+str(psnr_espcn)+' SSIM:'+str(ssim_espcn))
    print('       SRGAN PSNR:'+str(psnr_srgan)+' SSIM:'+str(ssim_srgan))
print('Average:')
print('     RESIZE PSNR:'+str(psnr_r_set5/5)+' SSIM:'+str(ssim_r_set5/5))
print('      SRCNN PSNR:'+str(psnr_srcnn_set5/5)+' SSIM:'+str(ssim_srcnn_set5/5))
print('      ESPCN PSNR:'+str(psnr_espcn_set5/5)+' SSIM:'+str(ssim_espcn_set5/5))
print('      SRGAN PSNR:'+str(psnr_srgan_set5/5)+' SSIM:'+str(ssim_srgan_set5/5))
    
print('======================================================================')    
img_path_set14 = '../dataset/set14'
f2 = os.listdir(img_path_set14)
ssim_r_set14 = 0
psnr_r_set14 = 0
ssim_srcnn_set14 = 0
psnr_srcnn_set14 = 0
ssim_espcn_set14 = 0
psnr_espcn_set14 = 0
ssim_srgan_set14 = 0
psnr_srgan_set14 = 0
for image_file in f2:
    
    image_in,image_lr,image_hr = img_preprocess(img_path_set14+'/'+image_file)
    
    ####RESIZE ONLY
    psnr_resize = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*image_in.squeeze(0).cpu().numpy().transpose([1, 2, 0]))
    ssim_resize = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*image_in.squeeze(0).cpu().numpy().transpose([1, 2, 0]))
    psnr_r_set14 += psnr_resize
    ssim_r_set14 += ssim_resize
    ####SRCNN
    result_SRCNN = model_SRCNN(image_in).clamp(0.0, 1.0)
    psnr_srcnn = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRCNN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_srcnn = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRCNN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_srcnn_set14 += psnr_srcnn
    ssim_srcnn_set14 += ssim_srcnn
    ####ESPCN
    result_ESPCN = model_ESPCN(image_lr).clamp(0.0, 1.0)
    psnr_espcn = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_ESPCN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_espcn = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_ESPCN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_espcn_set14 += psnr_espcn
    ssim_espcn_set14 += ssim_espcn
    ####SRGAN
    result_SRGAN = model_SRGAN(image_lr).clamp(0.0, 1.0)
    psnr_srgan = PSNR(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRGAN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    ssim_srgan = SSIM(255*image_hr.squeeze(0).cpu().numpy().transpose([1, 2, 0]),255*result_SRGAN.squeeze(0).cpu().detach().numpy().transpose([1, 2, 0]))
    psnr_srgan_set14 += psnr_srgan
    ssim_srgan_set14 += ssim_srgan
    
    
    print('Set5: '+image_file)
    print('      RESIZE PSNR:'+str(psnr_resize)+' SSIM:'+str(ssim_resize))
    print('       SRCNN PSNR:'+str(psnr_srcnn)+' SSIM:'+str(ssim_srcnn))
    print('       ESPCN PSNR:'+str(psnr_espcn)+' SSIM:'+str(ssim_espcn))
    print('       SRGAN PSNR:'+str(psnr_srgan)+' SSIM:'+str(ssim_srgan))
print('Average:')
print('     RESIZE PSNR:'+str(psnr_r_set14/14)+' SSIM:'+str(ssim_r_set14/14))
print('      SRCNN PSNR:'+str(psnr_srcnn_set14/14)+' SSIM:'+str(ssim_srcnn_set14/14))
print('      ESPCN PSNR:'+str(psnr_espcn_set14/14)+' SSIM:'+str(ssim_espcn_set14/14))
print('      SRGAN PSNR:'+str(psnr_srgan_set14/14)+' SSIM:'+str(ssim_srgan_set14/14))      


Set5: img_001.png
      RESIZE PSNR:35.13421450709595 SSIM:0.9079971135147644
       SRCNN PSNR:37.5565918757877 SSIM:0.951618363731478
       ESPCN PSNR:36.007899790392614 SSIM:0.9483355041247924
       ESPCN PSNR:30.920143944397363 SSIM:0.9036492704549086
Set5: img_002.png
      RESIZE PSNR:35.616507274301796 SSIM:0.922528782375049
       SRCNN PSNR:36.77142214750398 SSIM:0.9436271011660144
       ESPCN PSNR:35.53241824326744 SSIM:0.9235352180666399
       ESPCN PSNR:30.56701317120716 SSIM:0.8688554103046215
Set5: img_003.png
      RESIZE PSNR:28.82309668871239 SSIM:0.8787062056190916
       SRCNN PSNR:30.909944167358454 SSIM:0.9193094153735953
       ESPCN PSNR:30.32859914330903 SSIM:0.9061033391940184
       ESPCN PSNR:27.156334891586592 SSIM:0.8654922418165132
Set5: img_004.png
      RESIZE PSNR:35.1741416578918 SSIM:0.8805180983223974
       SRCNN PSNR:36.83169231990497 SSIM:0.925014151440218
       ESPCN PSNR:34.76954145019376 SSIM:0.8929336612114488
       ESPCN PSNR:31.6539359

--------- odict_items([('first_part', Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Tanh()
  (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Tanh()
)), ('last_part', Sequential(
  (0): Conv2d(32, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): PixelShuffle(upscale_factor=8)
))])
first_part
Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Tanh()
  (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Tanh()
)
name first_part
last_part
Sequential(
  (0): Conv2d(32, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): PixelShuffle(upscale_factor=8)
)
name last_part
torch.Size([1, 32, 200, 250])
